# RANDOM SEARCH HYPERPARAMETER OPTIMIZATION
## Model Klasifikasi Aflatoksin dengan ResNet-50

### Referensi Ilmiah:
1. **Bergstra & Bengio (2012)** - "Random Search for Hyper-Parameter Optimization"
2. **Yang & Shami (2020)** - "On Hyperparameter Optimization of Machine Learning Algorithms"
3. **Yu & Zhu (2020)** - "Hyper-Parameter Optimization: A Review of Algorithms and Applications"

In [ ]:
import os
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime
import pathlib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# REPRODUCIBILITY
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

In [ ]:
# KONFIGURASI
BASE_DIR = pathlib.Path(".")
DATA_DIR = BASE_DIR / 'dataset_final'  # Sesuaikan dengan folder Anda
RESULTS_DIR = BASE_DIR / 'random_search_results'
RESULTS_DIR.mkdir(exist_ok=True)

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
NUM_CLASSES = 4
AUTOTUNE = tf.data.AUTOTUNE

# Jumlah trial random search
N_TRIALS = 20

## SEARCH SPACE DEFINITION

Berdasarkan literatur ilmiah:

| Parameter | Range | Referensi |
|-----------|-------|----------|
| LR Phase 1 | [1e-5, 1e-3] | Kornblith et al. (2019) |
| LR Phase 2 | [1e-6, 1e-4] | Howard & Ruder (2018) |
| Dropout | [0.2, 0.5] | Srivastava et al. (2014) |
| Dense Units | [128, 1024] | He et al. (2016) |
| Fine-tune Layer | [100, 170] | Raghu et al. (2019) |

In [ ]:
# SEARCH SPACE
SEARCH_SPACE = {
    'lrp1': [1e-5, 5e-5, 1e-4, 5e-4, 1e-3],
    'lrp2': [1e-6, 5e-6, 1e-5, 5e-5, 1e-4],
    'layer_options': [
    [256],               # 1 layer:  256
    [512, 256],          # 2 layers: 512 → 256
    [256, 128],          # 2 layers: 256 → 128
    [512, 256, 64]],     # 3 layers: 512 → 256 → 64]
    'dropout_rate': [0.3, 0.4, 0.5, 0.6],
    'optimizer': ['adam', 'rmsprop'],
    'epochs_p1': [10, 15, 20, 25, 30, 35],
    'epochs_p2': [20, 30, 40, 50],
    'fine_tune_at': [140, 150, 160],
}

print("Search Space:")
for key, values in SEARCH_SPACE.items():
    print(f"  {key}: {values}")

In [ ]:
# UTILITY FUNCTIONS
def sample_hyperparameters(search_space, seed=None):
    """Random sampling dari search space."""
    if seed is not None:
        random.seed(seed)
    
    params = {}
    for key, values in search_space.items():
        params[key] = random.choice(values)
    
    # Constraint: lrp2 <= lrp1
    if params['lrp2'] > params['lrp1']:
        params['lrp2'] = params['lrp1'] / 10
    
    return params

def get_optimizer(name, learning_rate):
    """Create optimizer instance."""
    if name == 'adam':
        return keras.optimizers.Adam(learning_rate=learning_rate)
    elif name == 'adamw':
        return keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=1e-4)
    elif name == 'sgd':
        return keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    elif name == 'rmsprop':
        return keras.optimizers.RMSprop(learning_rate=learning_rate)
    return keras.optimizers.Adam(learning_rate=learning_rate)

# Test sampling - verifikasi bahwa arsitektur selalu terurut
print("\nContoh sampling (verifikasi urutan):")
for i in range(6):
    p = sample_hyperparameters(SEARCH_SPACE, seed=100+i)
    arch = " → ".join(map(str, p['layer_options']))
    print(f"  Sample {i+1}: [{arch}] ✓ Guaranteed descending")

In [ ]:
# LOAD DATASETS
print("Loading datasets...")

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR / 'train', labels="inferred", label_mode="int",
    image_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, shuffle=True, seed=42
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR / 'val', labels="inferred", label_mode="int",
    image_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, shuffle=False
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR / 'test', labels="inferred", label_mode="int",
    image_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, shuffle=False
)

class_names = train_ds.class_names
print(f"Classes: {class_names}")

# Augmentation & Preprocessing
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.028),
])

def preprocess_train(images, labels):
    images = tf.cast(images, tf.float32)
    images = data_augmentation(images, training=True)
    images = preprocess_input(images)
    return images, labels

def preprocess_val(images, labels):
    images = tf.cast(images, tf.float32)
    images = preprocess_input(images)
    return images, labels

train_ds = train_ds.map(preprocess_train, num_parallel_calls=AUTOTUNE).cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.map(preprocess_val, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)
test_ds = test_ds.map(preprocess_val, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)

print(f"✓ Data loaded")

In [ ]:
# BUILD MODEL FUNCTION
def build_model(params):
    """Build model dengan hyperparameters."""
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base_model.trainable = False
    
    inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dense layers berdasarkan layer_options
    layer_units = params['layer_options']  # Contoh: [512, 256, 64]
    
    for i, units in enumerate(layer_units):
        x = layers.Dense(units, activation='relu', name=f'dense_{i+1}')(x)
        # Dropout setelah setiap layer kecuali layer terakhir
        if i < len(layer_units) - 1:
            x = layers.Dropout(params['dropout_rate'], name=f'dropout_{i+1}')(x)
    
    # Jika hanya 1 layer, tambah dropout sebelum output
    if len(layer_units) == 1:
        x = layers.Dropout(params['dropout_rate'], name='dropout_1')(x)
    
    # Output
    outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='output')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model, base_model

In [ ]:
# TRAIN SINGLE TRIAL
def train_trial(params, trial_id):
    """Train single trial."""
    print(f"\n{'='*60}")
    print(f"TRIAL {trial_id}")
    print(f"{'='*60}")
    
    # Format architecture string
    layer_units = params['layer_options']
    arch_str = " → ".join(map(str, layer_units))
    
    print(f"Architecture: [{arch_str}] ({len(layer_units)} layers)")
    print(f"LR_P1={params['lrp1']}, LR_P2={params['lrp2']}, Dropout={params['dropout_rate']}")
    print(f"Optimizer={params['optimizer']}, Fine-tune at={params['fine_tune_at']}")
    
    model, base_model = build_model(params)
    
    callbacks = [
        keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, mode='max', restore_best_weights=True, verbose=0),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8, verbose=0)
    ]
    
    # Phase 1: Frozen backbone
    print(f"\n[Phase 1] Epochs={params['epochs_p1']}")
    model.compile(optimizer=get_optimizer(params['optimizer'], params['lrp1']),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history_p1 = model.fit(train_ds, validation_data=val_ds, epochs=params['epochs_p1'], callbacks=callbacks, verbose=0)
    p1_acc = max(history_p1.history['val_accuracy'])
    print(f"  Phase 1 Val Acc: {p1_acc:.4f}")
    
    # Phase 2: Fine-tuning
    print(f"\n[Phase 2] Epochs={params['epochs_p2']}, Fine-tune from layer {params['fine_tune_at']}")
    base_model.trainable = True
    for layer in base_model.layers[:params['fine_tune_at']]:
        layer.trainable = False
    
    model.compile(optimizer=get_optimizer(params['optimizer'], params['lrp2']),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    initial_epoch = len(history_p1.history['loss'])
    history_p2 = model.fit(train_ds, validation_data=val_ds, 
                           initial_epoch=initial_epoch, epochs=initial_epoch + params['epochs_p2'],
                           callbacks=callbacks, verbose=0)
    p2_acc = max(history_p2.history['val_accuracy'])
    print(f"  Phase 2 Val Acc: {p2_acc:.4f}")
    
    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)
    y_pred = np.argmax(model.predict(test_ds, verbose=0), axis=1)
    y_true = np.concatenate([y for x, y in test_ds], axis=0)
    f1 = f1_score(y_true, y_pred, average='macro')
    
    print(f"\n[Result] Test Accuracy: {test_accuracy:.4f}, F1 Macro: {f1:.4f}")
    
    return {
        'trial_id': trial_id,
        'architecture': arch_str,
        'params': params,
        'test_accuracy': float(test_accuracy),
        'test_loss': float(test_loss),
        'f1_macro': float(f1),
        'val_acc_p1': float(p1_acc),
        'val_acc_p2': float(p2_acc)
    }, model

In [ ]:
# RUN RANDOM SEARCH
print("="*70)
print("STARTING RANDOM SEARCH")
print("="*70)

all_results = []
best_result = None
best_accuracy = 0.0

for trial_id in range(1, N_TRIALS + 1):
    # Sample hyperparameters
    params = sample_hyperparameters(SEARCH_SPACE, seed=42 + trial_id)
    
    try:
        result, model = train_trial(params, trial_id)
        all_results.append(result)
        
        # Save trial
        with open(RESULTS_DIR / f'trial_{trial_id:03d}.json', 'w') as f:
            json.dump(result, f, indent=2)
        
        # Check if best
        if result['test_accuracy'] > best_accuracy:
            best_accuracy = result['test_accuracy']
            best_result = result
            model.save(RESULTS_DIR / 'best_model.keras')
            with open(RESULTS_DIR / 'best_params.json', 'w') as f:
                json.dump(result, f, indent=2)
            print(f"\n🏆 NEW BEST! Accuracy: {best_accuracy:.4f}")
        
        # Clear memory
        del model
        tf.keras.backend.clear_session()
        
    except Exception as e:
        print(f"Trial {trial_id} failed: {e}")
    
    print(f"\nProgress: {trial_id}/{N_TRIALS}, Best so far: {best_accuracy:.4f}")

# Save all results
with open(RESULTS_DIR / 'search_history.json', 'w') as f:
    json.dump(all_results, f, indent=2)

In [ ]:
# FINAL SUMMARY
print("\n" + "="*70)
print("RANDOM SEARCH COMPLETED")
print("="*70)

print(f"\nTotal trials: {len(all_results)}")
print(f"Best Test Accuracy: {best_accuracy:.4f}")

print("\n✅ BEST HYPERPARAMETERS:")
print("-"*40)
if best_result:
    print(f"  Architecture: [{best_result['architecture']}]")
    print(f"  LR Phase 1: {best_result['params']['lrp1']}")
    print(f"  LR Phase 2: {best_result['params']['lrp2']}")
    print(f"  Dropout: {best_result['params']['dropout_rate']}")
    print(f"  Optimizer: {best_result['params']['optimizer']}")
    print(f"  Epochs P1: {best_result['params']['epochs_p1']}")
    print(f"  Epochs P2: {best_result['params']['epochs_p2']}")
    print(f"  Fine-tune at: {best_result['params']['fine_tune_at']}")

# Sorted results table
print("\n📊 ALL TRIALS (sorted by accuracy):")
print("-"*70)
sorted_results = sorted(all_results, key=lambda x: x['test_accuracy'], reverse=True)
for r in sorted_results[:10]:
    print(f"Trial {r['trial_id']:3d}: Acc={r['test_accuracy']:.4f}, F1={r['f1_macro']:.4f}, "
          f"Arch=[{r['architecture']}]")

print(f"\n✅ Results saved to: {RESULTS_DIR}")